<div class="alert alert-info" style="color:black" > <h1> Activity 4: Creating a test harness for comparing ML algorithms on a dataset</h1>
<p> Now you have done some manual experimenting with different hyper-parameter values for algorithms, it's time to think about automating that process.</p>
<p>Complete the cell below to create a method that: </p>
<ul>
    <li> Takes a train and test data  arrays as  parameters <br>
        HINT: develop your code using train_x, test_x,train_y,test_y for the iris data from above</li>
    <li> Runs your SimpleKNNClassifier with K={1,3,5,7,9} and stores the test accuracy for each <br>
    HINT: you could use:
        <ul>
            <li>a for loop to run the algorithm with different settings k  for  the number of neighbours(K),</li>
            <li> an <a href=https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/>f-string</a> e.g. <code>experiment_name= f'KNN_K={k}'</code> to create a meaningful name for each run </li>
            <li>a   dictionary to store your results, where each experiment has the string <em>experiment_name</em> as the key and the accuracy as the value </li>
            </ul>for this?</li>
    <li> Runs a DecisionTreeClassifier with all the different combinations of hyper-parameters from activity 3<br>
       HINT: You could do this in the same way as I've suggested above but with nested for-loops (one for each hyper-parameter) and a more complex python f-string to create the name (key), then store the results in the same dictionary.  </li>
    <li> Reports the results and which algorithm-hyperparameter combination has the highest test accuracy</li>
</ul>
</div>

<div class="alert alert-warning" style="color:black"">
    <h3> Reminder: Storing data in python dictionaries and iterating through their contents</h3>
    <p> Python dictionaries are a way of storing data that can be accessed via a key<br>
for example: <code> my_dict= {'name':'jim','familyname':"Smith", 'job':'professor'}</code><br>
<b>Keys are usually strings</b>, but the values associated with a key can be any type, including numbers.</p>

<p> The following snippets of code might be useful to you - <b>after</b> you have edited them.</p>
<p> Make a new code cell in the notebook, copy the snippets in and run it, then edit it as you need.</p>
<p><pre style='background:lightbrown;colour:black'>    
labels = ['a','b','a','c','a','d','b']
indexes = [1,4,6]
mydict={}
<span style="color:green">for</span> idx <span style="color:green">in</span> indexes:
    <span style="color:green">if</span> labels[idx] <span style="color:green">in</span> mydict.keys():
        mydict[labels[idx]] += 1
    <span style="color:green">else</span>: #create a new dictionary entry if needed
        mydict[labels[idx]] = 1
<span style="color:green">print</span>(f'mydict is {mydict}')

leastvotes=99
<span style="color:green">for</span> key,val <span style="color:green">in</span> mydict.items():
    <span style="color:green">if</span> val < leastvotes:
        unpopular= key
        leastvotes=val
<span style="color:green">print</span>(f'{unpopular}, {leastvotes}')
    </pre></p>
    </div>

In [1]:
import numpy as np

#some names
people= ['jim', 'nathan']
RUNS= 5
#some randomly created scores
scores= [[46,23,69,12,78], [79, 13,48,63,39]]

#some example of making keys and putting things into  dict
mydict={}
#populate mydict with items created using the data above
for iteration in range(RUNS):
    for person_idx in range(2):
        name= f'{people[person_idx]}_{iteration}'
        mydict[name]= scores[person_idx][iteration]

#some examples of accessing a dictionary's contents
for key,val in mydict.items():
    print( f'{key} : {val}')

# print all scores
vals= mydict.values()
print(f' mydict.values() returns {vals} of type {type(vals)}')

# Have to cast vals into a list, then make a numpy array from that, to do maths
num_vals= np.array(list(vals))
print(f' biggest value is {np.max(num_vals)}') 

#print all keys
print(mydict.keys())


jim_0 : 46
nathan_0 : 79
jim_1 : 23
nathan_1 : 13
jim_2 : 69
nathan_2 : 48
jim_3 : 12
nathan_3 : 63
jim_4 : 78
nathan_4 : 39
 mydict.values() returns dict_values([46, 79, 23, 13, 69, 48, 12, 63, 78, 39]) of type <class 'dict_values'>
 biggest value is 79
dict_keys(['jim_0', 'nathan_0', 'jim_1', 'nathan_1', 'jim_2', 'nathan_2', 'jim_3', 'nathan_3', 'jim_4', 'nathan_4'])


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

class Simple1NNClassifier:
    def dist_a_b(self, a: np.array, b: np.array) -> float:
        return np.linalg.norm(a - b)
    
    def fit(self, x: np.ndarray, y: np.array):
        self.model_x = x
        self.model_y = y
    
    def predict(self, new_items: np.ndarray):
        num_new = new_items.shape[0]
        num_stored = self.model_x.shape[0]
        y_pred = np.zeros(num_new, dtype=int)
        distances = np.zeros((num_new, num_stored))
        for row, new_item in enumerate(new_items):
            for col, stored_example in enumerate(self.model_x):
                distances[row][col] = self.dist_a_b(new_item, stored_example)
            y_pred[row] = self.predict_one(row, distances)
        return y_pred
    
    def predict_one(self, item_idx: int, distances: np.ndarray):
        idx_of_nearest_neighbour = np.argmin(distances[item_idx])
        return self.model_y[idx_of_nearest_neighbour]

class SimpleKNNClassifier(Simple1NNClassifier):
    def __init__(self, K=1):
        self.K = K
    
    def predict_one(self, item_idx: int, distances: np.ndarray):
        nearest_indices = np.argsort(distances[item_idx])[:self.K]
        neighbor_labels = self.model_y[nearest_indices]
        label_counts = np.bincount(neighbor_labels, minlength=3)  # 3 classes in Iris
        return np.argmax(label_counts)

# Load Iris dataset
iris_data = load_iris()
irisX = iris_data.data
irisy = iris_data.target

# Split the data (using train_x, train_y, test_x, test_y as suggested)
train_x, test_x, train_y, test_y = train_test_split(irisX, irisy, test_size=0.33, 
                                                    stratify=irisy, random_state=42)

def first_ml_test_harness(train_x:np.ndarray,train_y:np.ndarray,
                          test_x:np.ndarray,test_y:np.ndarray):
    """ code to compare supervised machine learning algorithms on a dataset"""
    # insert your code below here
    
    results = {}
    
    # Testing SimpleKNNClassifier with K=[1, 3, 5, 7, 9]
    for k in [1, 3, 5, 7, 9]:
        experiment_name = f"KNN_K={k}"
        knn = SimpleKNNClassifier(K=k)
        knn.fit(train_x, train_y)
        predictions = knn.predict(test_x)
        accuracy = 100 * (test_y == predictions).sum() / test_y.shape[0]
        results[experiment_name] = accuracy
    
    max_depth_values = [None, 3, 5]
    min_samples_split_values = [2, 5, 10]
    
    for max_depth in max_depth_values:
        for min_samples_split in min_samples_split_values:
            experiment_name = f"DT_max_depth={max_depth}_min_samples_split={min_samples_split}"
            dt = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, random_state=42)
            dt.fit(train_x, train_y)
            predictions = dt.predict(test_x)
            accuracy = 100 * (test_y == predictions).sum() / test_y.shape[0]
            results[experiment_name] = accuracy
    
    
   # Reports results
    print("\n=== Test Harness Results ===")
    for experiment_name, accuracy in results.items():
        print(f"{experiment_name}: Test Accuracy = {accuracy:.2f}%")
    
    # Findingg the best performing model
    best_experiment = max(results, key=results.get)
    best_accuracy = results[best_experiment]
    print(f"\nBest Model: {best_experiment} with Test Accuracy = {best_accuracy:.2f}%")
    
    return results

    #insert your code above here

In [3]:
#now run your code for the iris data
results = first_ml_test_harness(train_x, train_y, test_x, test_y)



=== Test Harness Results ===
KNN_K=1: Test Accuracy = 94.00%
KNN_K=3: Test Accuracy = 96.00%
KNN_K=5: Test Accuracy = 98.00%
KNN_K=7: Test Accuracy = 98.00%
KNN_K=9: Test Accuracy = 96.00%
DT_max_depth=None_min_samples_split=2: Test Accuracy = 94.00%
DT_max_depth=None_min_samples_split=5: Test Accuracy = 94.00%
DT_max_depth=None_min_samples_split=10: Test Accuracy = 94.00%
DT_max_depth=3_min_samples_split=2: Test Accuracy = 98.00%
DT_max_depth=3_min_samples_split=5: Test Accuracy = 98.00%
DT_max_depth=3_min_samples_split=10: Test Accuracy = 98.00%
DT_max_depth=5_min_samples_split=2: Test Accuracy = 94.00%
DT_max_depth=5_min_samples_split=5: Test Accuracy = 94.00%
DT_max_depth=5_min_samples_split=10: Test Accuracy = 94.00%

Best Model: KNN_K=5 with Test Accuracy = 98.00%
